In [3]:
import pyproj
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.transform import from_origin
import numpy as np


def lat_long_to_bng(lat, long):
    wgs84 = pyproj.CRS('EPSG:4326')  # WGS84坐标系（经纬度）
    bng = pyproj.CRS('EPSG:27700')   # British National Grid坐标系

    transformer = pyproj.Transformer.from_crs(wgs84, bng, always_xy=True)

    bng_x, bng_y = transformer.transform(long, lat)

    return bng_x, bng_y


In [4]:
# 读取Excel文件
df = pd.read_csv("/Users/fionamac/Documents/非文本数据提取/classification——output2.csv")



In [5]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat']), crs='EPSG:4326')



In [6]:
# 将GeoDataFrame中的经纬度坐标投影为British National Grid坐标
gdf = gdf.to_crs('EPSG:27700')

In [9]:
# 打开Tiff文件
tiff_file = 'prob_wildfire.tif'
with rasterio.open(tiff_file) as src:
    # 获取Tiff文件的变换信息和数据
    transform = src.transform
    data = src.read(1, masked=True)
    # 提取每个点的像素值
    pixel_values = []
    for index, row in gdf.iterrows():
        lon, lat = row.geometry.x, row.geometry.y
        col, row = ~transform * (lon, lat)  # 将经纬度坐标转换为像素坐标
        try:
            value = data[int(row)][int(col)]
        except Exception as e:
            value = np.nan
        pixel_values.append(value)
        
        
         # 将像素值添加到GeoDataFrame中
    gdf['pixel_value'] = pixel_values

# 打印包含像素值的GeoDataFrame
print(gdf)

     resource_callsign callsign_type    incident_number  \
0                 B041           PRL  FW001692-09012023   
1                 C085           BRV  FW001696-09012023   
2                 C091           PRL  FW001696-09012023   
3                 C095           BRV  FW001696-09012023   
4                 C095           BRV  FW001697-09012023   
...                ...           ...                ...   
4415              D095           BRV  FW018273-22032023   
4416              E012           PRL  FW018279-22032023   
4417              C071           PRL  FW018285-22032023   
4418              C012           PRL  FW018290-22032023   
4419              C071           PRL  FW018302-23032023   

                    call_time revised_incident_type            assigned_time  \
0     2023-01-09 18:21:36.333                   RTC  2023-01-09 18:23:15.637   
1     2023-01-09 18:45:04.243                   RTC  2023-01-09 18:46:15.830   
2     2023-01-09 18:45:04.243                   RTC

In [10]:
# 将 GDF 导出为 CSV 文件
gdf.to_csv('output_with_fire_prob.csv', index=False)

In [12]:

gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4420 entries, 0 to 4419
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   resource_callsign      4420 non-null   object  
 1   callsign_type          4420 non-null   object  
 2   incident_number        4420 non-null   object  
 3   call_time              4420 non-null   object  
 4   revised_incident_type  4420 non-null   object  
 5   assigned_time          4420 non-null   object  
 6   mobile_time            4357 non-null   object  
 7   on_scene_time          4175 non-null   object  
 8   available_time         4402 non-null   object  
 9   callsign_station       4420 non-null   object  
 10  alias_1                4420 non-null   object  
 11  initial_incident_type  4420 non-null   object  
 12  initial_role           941 non-null    object  
 13  final_role             4403 non-null   object  
 14  actual_role            4420 non-

In [19]:
# 读取Excel文件
df = pd.read_csv("output_with_fire_prob.csv")



In [20]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat']), crs='EPSG:4326')



In [21]:
# 将GeoDataFrame中的经纬度坐标投影为British National Grid坐标
gdf = gdf.to_crs('EPSG:27700')

In [22]:
# 打开Tiff文件
tiff_file = 'weighted_sum2.tif'
with rasterio.open(tiff_file) as src:
    # 获取Tiff文件的变换信息和数据
    transform = src.transform
    data = src.read(1, masked=True)
    # 提取每个点的像素值
    pixel_values = []
    for index, row in gdf.iterrows():
        lon, lat = row.geometry.x, row.geometry.y
        col, row = ~transform * (lon, lat)  # 将经纬度坐标转换为像素坐标
        try:
            value = data[int(row)][int(col)]
        except Exception as e:
            value = np.nan
        pixel_values.append(value)
        
        
         # 将像素值添加到GeoDataFrame中
    gdf['suitability_or'] = pixel_values

# 打印包含像素值的GeoDataFrame
print(gdf)

     resource_callsign callsign_type    incident_number  \
0                 B041           PRL  FW001692-09012023   
1                 C085           BRV  FW001696-09012023   
2                 C091           PRL  FW001696-09012023   
3                 C095           BRV  FW001696-09012023   
4                 C095           BRV  FW001697-09012023   
...                ...           ...                ...   
4415              D095           BRV  FW018273-22032023   
4416              E012           PRL  FW018279-22032023   
4417              C071           PRL  FW018285-22032023   
4418              C012           PRL  FW018290-22032023   
4419              C071           PRL  FW018302-23032023   

                    call_time revised_incident_type            assigned_time  \
0     2023-01-09 18:21:36.333                   RTC  2023-01-09 18:23:15.637   
1     2023-01-09 18:45:04.243                   RTC  2023-01-09 18:46:15.830   
2     2023-01-09 18:45:04.243                   RTC

In [24]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4420 entries, 0 to 4419
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   resource_callsign      4420 non-null   object  
 1   callsign_type          4420 non-null   object  
 2   incident_number        4420 non-null   object  
 3   call_time              4420 non-null   object  
 4   revised_incident_type  4420 non-null   object  
 5   assigned_time          4420 non-null   object  
 6   mobile_time            4357 non-null   object  
 7   on_scene_time          4175 non-null   object  
 8   available_time         4402 non-null   object  
 9   callsign_station       4420 non-null   object  
 10  alias_1                4420 non-null   object  
 11  initial_incident_type  4420 non-null   object  
 12  initial_role           941 non-null    object  
 13  final_role             4403 non-null   object  
 14  actual_role            4420 non-

In [25]:
data = gdf 

In [26]:

# 将'pixel_value'和'suitability_or'列转换为数值类型，将无法转换的值设置为NaN
data['pixel_value'] = pd.to_numeric(data['pixel_value'], errors='coerce')
data['suitability_or'] = pd.to_numeric(data['suitability_or'], errors='coerce')

# 使用均值对NaN值进行插值
data['pixel_value'].fillna(data['pixel_value'].mean(), inplace=True)
data['suitability_or'].fillna(data['suitability_or'].mean(), inplace=True)


In [27]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4420 entries, 0 to 4419
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   resource_callsign      4420 non-null   object  
 1   callsign_type          4420 non-null   object  
 2   incident_number        4420 non-null   object  
 3   call_time              4420 non-null   object  
 4   revised_incident_type  4420 non-null   object  
 5   assigned_time          4420 non-null   object  
 6   mobile_time            4357 non-null   object  
 7   on_scene_time          4175 non-null   object  
 8   available_time         4402 non-null   object  
 9   callsign_station       4420 non-null   object  
 10  alias_1                4420 non-null   object  
 11  initial_incident_type  4420 non-null   object  
 12  initial_role           941 non-null    object  
 13  final_role             4403 non-null   object  
 14  actual_role            4420 non-

In [31]:
# 尝试将'suitability_or'列转换为浮点数，无法转换的值将被设置为NaN
data['suitability'] = pd.to_numeric(data['suitability_or'], errors='coerce')


In [32]:
data['suitability'].head()

0      --
1    3.83
2    3.83
3    3.83
4    3.51
Name: suitability, dtype: object

In [30]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4420 entries, 0 to 4419
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   resource_callsign      4420 non-null   object  
 1   callsign_type          4420 non-null   object  
 2   incident_number        4420 non-null   object  
 3   call_time              4420 non-null   object  
 4   revised_incident_type  4420 non-null   object  
 5   assigned_time          4420 non-null   object  
 6   mobile_time            4357 non-null   object  
 7   on_scene_time          4175 non-null   object  
 8   available_time         4402 non-null   object  
 9   callsign_station       4420 non-null   object  
 10  alias_1                4420 non-null   object  
 11  initial_incident_type  4420 non-null   object  
 12  initial_role           941 non-null    object  
 13  final_role             4403 non-null   object  
 14  actual_role            4420 non-

In [35]:

from sklearn.preprocessing import MinMaxScaler

# 创建MinMaxScaler对象
scaler = MinMaxScaler()
# 假设您的数据框中包含一个名为'suitability'的列
data['suitability2'] = scaler.fit_transform(data[['suitability']])


/opt/anaconda3/envs/envGEOG0051/lib/python3.10/site-packages/pandas/core/generic.py:2070: UserWarning: Warning: converting a masked element to nan.
  return np.asarray(self._values, dtype=dtype)
/opt/anaconda3/envs/envGEOG0051/lib/python3.10/site-packages/pandas/core/generic.py:2070: UserWarning: Warning: converting a masked element to nan.
  return np.asarray(self._values, dtype=dtype)


In [36]:
data.head()

,resource_callsign,callsign_type,incident_number,call_time,revised_incident_type,assigned_time,mobile_time,on_scene_time,available_time,callsign_station,...,mob_reason,lat,long,details,classification,geometry,pixel_value,suitability_or,suitability,suitability2
0,B041,PRL,FW001692-09012023,2023-01-09 18:21:36.333,RTC,2023-01-09 18:23:15.637,2023-01-09 18:23:42.477,2023-01-09 18:29:07.903,2023-01-09 19:01:05.753,Canley,...,Initial Response,52.389348,-1.599208,road traffic Collison FLUIDS LEAKING - NO REPO...,Class 4,POINT (427372.338 276887.244),0.650336,--,--,NaN
1,C085,BRV,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.830,2023-01-09 18:47:19.143,2023-01-09 18:50:40.103,2023-01-09 19:06:49.817,Billesley,...,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 2,POINT (411180.592 281672.376),0.589420,3.83,3.83,0.750000
2,C091,PRL,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.847,2023-01-09 18:47:29.960,2023-01-09 18:51:50.057,2023-01-09 19:07:35.297,Hay Mills,...,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 3,POINT (411180.592 281672.376),0.589420,3.83,3.83,0.750000
3,C095,BRV,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.877,2023-01-09 18:47:15.327,2023-01-09 18:52:11.197,2023-01-09 19:00:12.617,Hay Mills,...,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 2,POINT (411180.592 281672.376),0.589420,3.83,3.83,0.750000
4,C095,BRV,FW001697-09012023,2023-01-09 19:06:45.023,RTC,2023-01-09 19:08:08.300,2023-01-09 19:08:22.170,2023-01-09 19:08:25.217,2023-01-09 20:16:50.070,Hay Mills,...,Initial Response,52.438270,-1.835599,NOBODY TRAPPED - 2 CAR road traffic Collison ...,Class 2,POINT (411272.602 282266.374),0.803614,3.51,3.51,0.634058


In [39]:
data['suitability2'].fillna(data['suitability2'].mean(), inplace=True)

In [40]:
data['suitability2'].mean()

0.5199768343213695

In [38]:
data.head()

,resource_callsign,callsign_type,incident_number,call_time,revised_incident_type,assigned_time,mobile_time,on_scene_time,available_time,callsign_station,...,mob_reason,lat,long,details,classification,geometry,pixel_value,suitability_or,suitability,suitability2
0,B041,PRL,FW001692-09012023,2023-01-09 18:21:36.333,RTC,2023-01-09 18:23:15.637,2023-01-09 18:23:42.477,2023-01-09 18:29:07.903,2023-01-09 19:01:05.753,Canley,...,Initial Response,52.389348,-1.599208,road traffic Collison FLUIDS LEAKING - NO REPO...,Class 4,POINT (427372.338 276887.244),0.650336,--,--,NaN
1,C085,BRV,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.830,2023-01-09 18:47:19.143,2023-01-09 18:50:40.103,2023-01-09 19:06:49.817,Billesley,...,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 2,POINT (411180.592 281672.376),0.589420,3.83,3.83,0.750000
2,C091,PRL,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.847,2023-01-09 18:47:29.960,2023-01-09 18:51:50.057,2023-01-09 19:07:35.297,Hay Mills,...,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 3,POINT (411180.592 281672.376),0.589420,3.83,3.83,0.750000
3,C095,BRV,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.877,2023-01-09 18:47:15.327,2023-01-09 18:52:11.197,2023-01-09 19:00:12.617,Hay Mills,...,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 2,POINT (411180.592 281672.376),0.589420,3.83,3.83,0.750000
4,C095,BRV,FW001697-09012023,2023-01-09 19:06:45.023,RTC,2023-01-09 19:08:08.300,2023-01-09 19:08:22.170,2023-01-09 19:08:25.217,2023-01-09 20:16:50.070,Hay Mills,...,Initial Response,52.438270,-1.835599,NOBODY TRAPPED - 2 CAR road traffic Collison ...,Class 2,POINT (411272.602 282266.374),0.803614,3.51,3.51,0.634058


In [42]:
data['suitability2'].fillna(data['suitability2'].mean(), inplace=True)

In [44]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4420 entries, 0 to 4419
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   resource_callsign      4420 non-null   object  
 1   callsign_type          4420 non-null   object  
 2   incident_number        4420 non-null   object  
 3   call_time              4420 non-null   object  
 4   revised_incident_type  4420 non-null   object  
 5   assigned_time          4420 non-null   object  
 6   mobile_time            4357 non-null   object  
 7   on_scene_time          4175 non-null   object  
 8   available_time         4402 non-null   object  
 9   callsign_station       4420 non-null   object  
 10  alias_1                4420 non-null   object  
 11  initial_incident_type  4420 non-null   object  
 12  initial_role           941 non-null    object  
 13  final_role             4403 non-null   object  
 14  actual_role            4420 non-

In [45]:
data.head()

,resource_callsign,callsign_type,incident_number,call_time,revised_incident_type,assigned_time,mobile_time,on_scene_time,available_time,callsign_station,...,mob_reason,lat,long,details,classification,geometry,pixel_value,suitability_or,suitability,suitability2
0,B041,PRL,FW001692-09012023,2023-01-09 18:21:36.333,RTC,2023-01-09 18:23:15.637,2023-01-09 18:23:42.477,2023-01-09 18:29:07.903,2023-01-09 19:01:05.753,Canley,...,Initial Response,52.389348,-1.599208,road traffic Collison FLUIDS LEAKING - NO REPO...,Class 4,POINT (427372.338 276887.244),0.650336,--,--,0.519977
1,C085,BRV,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.830,2023-01-09 18:47:19.143,2023-01-09 18:50:40.103,2023-01-09 19:06:49.817,Billesley,...,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 2,POINT (411180.592 281672.376),0.589420,3.83,3.83,0.750000
2,C091,PRL,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.847,2023-01-09 18:47:29.960,2023-01-09 18:51:50.057,2023-01-09 19:07:35.297,Hay Mills,...,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 3,POINT (411180.592 281672.376),0.589420,3.83,3.83,0.750000
3,C095,BRV,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.877,2023-01-09 18:47:15.327,2023-01-09 18:52:11.197,2023-01-09 19:00:12.617,Hay Mills,...,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 2,POINT (411180.592 281672.376),0.589420,3.83,3.83,0.750000
4,C095,BRV,FW001697-09012023,2023-01-09 19:06:45.023,RTC,2023-01-09 19:08:08.300,2023-01-09 19:08:22.170,2023-01-09 19:08:25.217,2023-01-09 20:16:50.070,Hay Mills,...,Initial Response,52.438270,-1.835599,NOBODY TRAPPED - 2 CAR road traffic Collison ...,Class 2,POINT (411272.602 282266.374),0.803614,3.51,3.51,0.634058


In [46]:
# 使用drop方法删除'suitability_or'和'suitability'列
data = data.drop(columns=['suitability_or', 'suitability'])

In [48]:
data.head() 

,resource_callsign,callsign_type,incident_number,call_time,revised_incident_type,assigned_time,mobile_time,on_scene_time,available_time,callsign_station,...,riders,assign_order,mob_reason,lat,long,details,classification,geometry,pixel_value,suitability2
0,B041,PRL,FW001692-09012023,2023-01-09 18:21:36.333,RTC,2023-01-09 18:23:15.637,2023-01-09 18:23:42.477,2023-01-09 18:29:07.903,2023-01-09 19:01:05.753,Canley,...,5,1,Initial Response,52.389348,-1.599208,road traffic Collison FLUIDS LEAKING - NO REPO...,Class 4,POINT (427372.338 276887.244),0.650336,0.519977
1,C085,BRV,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.830,2023-01-09 18:47:19.143,2023-01-09 18:50:40.103,2023-01-09 19:06:49.817,Billesley,...,3,1,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 2,POINT (411180.592 281672.376),0.589420,0.750000
2,C091,PRL,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.847,2023-01-09 18:47:29.960,2023-01-09 18:51:50.057,2023-01-09 19:07:35.297,Hay Mills,...,4,2,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 3,POINT (411180.592 281672.376),0.589420,0.750000
3,C095,BRV,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.877,2023-01-09 18:47:15.327,2023-01-09 18:52:11.197,2023-01-09 19:00:12.617,Hay Mills,...,3,3,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 2,POINT (411180.592 281672.376),0.589420,0.750000
4,C095,BRV,FW001697-09012023,2023-01-09 19:06:45.023,RTC,2023-01-09 19:08:08.300,2023-01-09 19:08:22.170,2023-01-09 19:08:25.217,2023-01-09 20:16:50.070,Hay Mills,...,3,1,Initial Response,52.438270,-1.835599,NOBODY TRAPPED - 2 CAR road traffic Collison ...,Class 2,POINT (411272.602 282266.374),0.803614,0.634058


In [49]:
data = data.rename(columns={'suitability2': 'suitability'})

In [50]:
data.head()

,resource_callsign,callsign_type,incident_number,call_time,revised_incident_type,assigned_time,mobile_time,on_scene_time,available_time,callsign_station,...,riders,assign_order,mob_reason,lat,long,details,classification,geometry,pixel_value,suitability
0,B041,PRL,FW001692-09012023,2023-01-09 18:21:36.333,RTC,2023-01-09 18:23:15.637,2023-01-09 18:23:42.477,2023-01-09 18:29:07.903,2023-01-09 19:01:05.753,Canley,...,5,1,Initial Response,52.389348,-1.599208,road traffic Collison FLUIDS LEAKING - NO REPO...,Class 4,POINT (427372.338 276887.244),0.650336,0.519977
1,C085,BRV,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.830,2023-01-09 18:47:19.143,2023-01-09 18:50:40.103,2023-01-09 19:06:49.817,Billesley,...,3,1,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 2,POINT (411180.592 281672.376),0.589420,0.750000
2,C091,PRL,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.847,2023-01-09 18:47:29.960,2023-01-09 18:51:50.057,2023-01-09 19:07:35.297,Hay Mills,...,4,2,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 3,POINT (411180.592 281672.376),0.589420,0.750000
3,C095,BRV,FW001696-09012023,2023-01-09 18:45:04.243,RTC,2023-01-09 18:46:15.877,2023-01-09 18:47:15.327,2023-01-09 18:52:11.197,2023-01-09 19:00:12.617,Hay Mills,...,3,3,Initial Response,52.432931,-1.836972,road traffic Collison ROLLOVER CAR versus PARK...,Class 2,POINT (411180.592 281672.376),0.589420,0.750000
4,C095,BRV,FW001697-09012023,2023-01-09 19:06:45.023,RTC,2023-01-09 19:08:08.300,2023-01-09 19:08:22.170,2023-01-09 19:08:25.217,2023-01-09 20:16:50.070,Hay Mills,...,3,1,Initial Response,52.438270,-1.835599,NOBODY TRAPPED - 2 CAR road traffic Collison ...,Class 2,POINT (411272.602 282266.374),0.803614,0.634058


In [51]:
# 将 GDF 导出为 CSV 文件
gdf.to_csv('classification_with_GIS.csv', index=False)

In [52]:
pwd

'/Users/fionamac/Documents/非文本数据提取'